## GB and MLP model notebook

**This notebook contains the code for inputting data from training sets, applying preprocessing methods from modules, and running data through GB and MLP classifiers**

In [91]:
#imports

from sklearn.model_selection import train_test_split
from modules.classification_model_skeletons import *
from modules.data_preprocessing import *


In [92]:
df = load_data_into_df('data/IBM_Superposition_QRNG_100qubit.txt', ['quantum', 'QPU'])

In [93]:
df_testing = filter_by_label(df, ['binary_number', 'quantum'])

# run below line if classifying by QPU, otherwise comment out 
# (first 40000 lines are QRNG data, next 40000 lines are PRNG data)
# df_testing = df_testing[:40000]

df_testing

,binary_number,quantum
0,0001011100100011110100100001101111000011010101...,quantum
1,0110101010000001110100000100111101101111110111...,quantum
2,1001001010000000001111010010111000100011110000...,quantum
3,0101100101110010001001011000011010111011101101...,quantum
4,1000101001000000111110011000010111010011111110...,quantum
...,...,...
79995,0011111011100101010101000110001001011000010001...,non-quantum
79996,0001011001010010110001010111010010011110100000...,non-quantum
79997,1000001100011000110001111000010100100110111000...,non-quantum
79998,0101101010110011010011110011100011010000001101...,non-quantum


In [94]:
# For testing with different input lengths (base length = 100)

df_testing_concatenated = concatenate_data(df_testing, 80)
df_testing_concatenated

,Concatenated_Data,quantum
0,0001011100100011110100100001101111000011010101...,quantum
1,1101011100111101100101100110110110011101111111...,quantum
2,1000100100100000111111010101111011110001111010...,quantum
3,1111000001101101110001110111110001010011110101...,quantum
4,0110001111111010111100000001110001001110011000...,quantum
...,...,...
995,0011001001011101101001110111000001111010010011...,non-quantum
996,0000001101110000010000101000100000011111110010...,non-quantum
997,1000011001001001011100000011000111101100010110...,non-quantum
998,0010100010101001111011110110101010101101000010...,non-quantum


In [95]:
# Applying preprocessing feature extraction tests:
tests = ['counts']
'''tests = ['counts', 'runs', 'unq_subsq', 'longest_run']'''

# See modules directory for function documentation
df_testing_features = apply_individual_qubit_functions(df_testing_concatenated, tests, 100)
df_testing_features

/Users/shrey/Dev/qc-classifier-experiments/modules/data_preprocessing.py:116: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{function}_qb_{qb}'] = qubitRes
/Users/shrey/Dev/qc-classifier-experiments/modules/data_preprocessing.py:116: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{function}_qb_{qb}'] = qubitRes


,Concatenated_Data,quantum,counts_qb_0,counts_qb_1,counts_qb_2,counts_qb_3,counts_qb_4,counts_qb_5,counts_qb_6,counts_qb_7,...,counts_qb_90,counts_qb_91,counts_qb_92,counts_qb_93,counts_qb_94,counts_qb_95,counts_qb_96,counts_qb_97,counts_qb_98,counts_qb_99
0,0001011100100011110100100001101111000011010101...,quantum,34,43,41,34,36,52,38,35,...,37,42,39,39,47,36,34,32,33,45
1,1101011100111101100101100110110110011101111111...,quantum,45,33,43,34,33,35,42,49,...,36,41,43,36,40,41,39,47,44,41
2,1000100100100000111111010101111011110001111010...,quantum,44,41,41,22,40,39,34,31,...,42,43,35,39,38,43,37,40,39,37
3,1111000001101101110001110111110001010011110101...,quantum,36,41,37,36,46,38,41,35,...,43,33,36,26,42,43,39,47,43,42
4,0110001111111010111100000001110001001110011000...,quantum,44,31,46,43,40,31,42,48,...,35,43,45,47,40,42,36,42,37,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0011001001011101101001110111000001111010010011...,non-quantum,36,50,46,31,36,42,32,43,...,36,42,44,41,41,43,43,37,42,44
996,0000001101110000010000101000100000011111110010...,non-quantum,40,35,40,44,39,47,35,38,...,41,36,43,38,39,40,46,40,35,34
997,1000011001001001011100000011000111101100010110...,non-quantum,45,36,38,37,39,40,38,47,...,38,35,35,39,38,37,39,44,48,36
998,0010100010101001111011110110101010101101000010...,non-quantum,39,44,40,33,39,51,37,46,...,46,41,41,39,44,41,39,38,38,36


In [96]:
# For counting individual bits as features: if just using frequency 
# as the only feature, comment out to avoid redundancy: 

'''
df_testing_features = make_bit_features(df_testing_concatenated)
#df_testing_features
'''

df_testing_features = df_testing_features.drop(columns='Concatenated_Data')
df_testing_features

,quantum,counts_qb_0,counts_qb_1,counts_qb_2,counts_qb_3,counts_qb_4,counts_qb_5,counts_qb_6,counts_qb_7,counts_qb_8,...,counts_qb_90,counts_qb_91,counts_qb_92,counts_qb_93,counts_qb_94,counts_qb_95,counts_qb_96,counts_qb_97,counts_qb_98,counts_qb_99
0,quantum,34,43,41,34,36,52,38,35,47,...,37,42,39,39,47,36,34,32,33,45
1,quantum,45,33,43,34,33,35,42,49,43,...,36,41,43,36,40,41,39,47,44,41
2,quantum,44,41,41,22,40,39,34,31,37,...,42,43,35,39,38,43,37,40,39,37
3,quantum,36,41,37,36,46,38,41,35,41,...,43,33,36,26,42,43,39,47,43,42
4,quantum,44,31,46,43,40,31,42,48,40,...,35,43,45,47,40,42,36,42,37,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,non-quantum,36,50,46,31,36,42,32,43,36,...,36,42,44,41,41,43,43,37,42,44
996,non-quantum,40,35,40,44,39,47,35,38,45,...,41,36,43,38,39,40,46,40,35,34
997,non-quantum,45,36,38,37,39,40,38,47,43,...,38,35,35,39,38,37,39,44,48,36
998,non-quantum,39,44,40,33,39,51,37,46,46,...,46,41,41,39,44,41,39,38,38,36


In [97]:
# Splitting intro training and testing data: 80% and 20% split

X = df_testing_features.drop(columns=df_testing_features.columns[0]).values
y = df_testing_features[df_testing_features.columns[0]].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### GB model

In [98]:
accuracy = sk_learn_gradient_boosting(X_train, y_train, X_test, y_test)
print('GB accuracy: ', accuracy)

fitting model
model fitted, now making predictions on test data
GB accuracy:  0.955


### Basic MLP model 
**nodes in hidden layer = size of input layer (for consistency)**

In [99]:
num_features = df_testing_features.shape[1] - 1
accuracy = sk_learn_MLP(X_train, y_train, X_test, y_test, num_features)
print('MLP accuracy: ', accuracy)

fitting model
model fitted, now making predictions on test data
MLP accuracy:  0.935
